In [ ]:
# Import libraries. 
# Generate images in the notebook
%matplotlib inline

import matplotlib.pyplot as plt
import collections
from collections import defaultdict
import gzip
import itertools
import numpy as np
import os
import time
import pandas as pd
import seaborn as sns

import bokeh
import bokeh.io
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, save, output_notebook, output_file

# Import colour palettes for later on
from bokeh.palettes import Category20b
from bokeh.palettes import Purples
from bokeh.palettes import Greens
from bokeh.palettes import YlOrBr
from bokeh.palettes import YlOrRd
from bokeh.palettes import PuOr
from bokeh.palettes import RdGy

# Dimension reduction tools
from sklearn.decomposition import PCA as PCA
from sklearn.manifold import TSNE
import umap 

In [ ]:
#loading population data
population_names= pd.read_excel(r'sample_info.xls')
print(population_names)


In [ ]:
#Generating the pandas dataframe called Data_Struct
Data_Struct=population_names

In [ ]:
#loading the covariance matrix
#this list contains all covariance matrices as an exhaustive list
#it will be used to loop through
file_list=('dam1_20_maf01.txt','dam1_20_maf05.txt','dam1_50_maf01.txt','dam1_50_maf05.txt','dam1_100_maf01.txt'\
           ,'dam1_100_maf05.txt','mm1_20_maf01.txt','mm1_20_maf05.txt','mm1_50_maf01.txt','mm1_50_maf05.txt'\
           ,'mm1_100_maf01.txt','mm1_100_maf05.txt')        
for filename in file_list:


#filename=('mm1_50_maf01.txt') #this is to prevent it won't run through for loop for 12 files.
#i=1
#while i < 2:
#    i+=1
    

    
    cov_mat= pd.read_csv(filename, sep=' ', header=None)
    cov_mat_np=cov_mat.to_numpy()

    # calculating eigen vectors and eigen values from the initial covariance matrix
    eigen_vals, eigen_vecs = np.linalg.eig(cov_mat_np)

    # sorting them from largest to smallest
    idx = eigen_vals.argsort()[::-1]   
    eigenValues = eigen_vals[idx]
    eigenVectors = eigen_vecs[:,idx]
    eigenVectors_real=eigenVectors.real # remeving the imaginary part of the eigen vectors
    
    # calculating the weighted PCA
    eigvec_mltply_val=eigenVectors.real*eigenValues.real 

    # calculating the total explained variance
    expl_pre=eigenValues/sum(eigenValues)
    expl=np.cumsum(expl_pre)

    expl_df=pd.DataFrame(expl_pre*100,columns=['explained_variance'])
    expl_df['cumulative_expl']=expl*100
    expl_df.set_index(np.arange(1,144))


    Data_Struct['EigenVect1']=eigenVectors_real[:,0]
    Data_Struct['EigenVect2']=eigenVectors_real[:,1]
    Data_Struct['wEigenVect1']=eigvec_mltply_val.real[:,0]
    Data_Struct['wEigenVect2']=eigvec_mltply_val.real[:,1]

    # Number of principal components to use in a for loop
    number_of_PCs_list=(15,20,25,30,35,40)
    #number_of_PCs_list=(15,20)
    
    for n_pc in number_of_PCs_list:

    
#    n_pc=15 # this is to prevent the for loop and calculate a single value as a test-drive
#    while n_pc <20:
#        n_pc+=10
    

        
        # Project the wPCA(eigenvectors*eigenvalues) via t-SNE to 2 dimensions.
        perplexity_values=(5,10,30,50)
        for perp in perplexity_values:
            np.random.seed(111)
            proj_tsne = TSNE(n_components=2,perplexity=perp).fit_transform(eigvec_mltply_val[:,:n_pc])
            Data_Struct['tSNE-1 perp'+str(perp)]=proj_tsne[:,0]
            Data_Struct['tSNE-2 perp'+str(perp)]=proj_tsne[:,1]
        
        
        
        # Project the wPCA(eigenvectors*eigenvalues) via UMAPto 2 dimensions.        
        n_neighbors_nums=(5,10,15,20)
        mindists=(0.001,0.01,0.1,0.5)
        
        for nn in n_neighbors_nums:
            for mind in mindists:
                np.random.seed(111)
                proj_umap = umap.UMAP(n_components=2, n_neighbors=nn, min_dist=mind).fit_transform(eigvec_mltply_val[:,:n_pc])
                Data_Struct['UMAP-1 numn'+str(nn)+' mindist'+str(mind)]=proj_umap[:,0]
                Data_Struct['UMAP-2 numn'+str(nn)+' mindist'+str(mind)]=proj_umap[:,1]

        # plotting part
        fig, axs = plt.subplots(4, 5,figsize=(25, 20))

        #axs[1,1] = Data_Struct.plot.scatter(x='X1',y='X2',c='Population')


        fig.suptitle("input:"+filename+' / Top '+str(n_pc)+'PCs are used ('+str(round(expl_df['cumulative_expl'][n_pc-1],1))+'%)', fontsize=14)

        sns.scatterplot(ax=axs[0,0],data=Data_Struct, x='tSNE-1 perp5', y='tSNE-2 perp5', hue='Population',legend = False)#.set(title='PCA')
        axs[0, 0].set_title('tSNE / Perplexity = '+str(perplexity_values[0]))
        axs[0, 0].set_xlabel('tSNE-1')
        axs[0, 0].set_ylabel('tSNE-2')
        axs[0, 0].set_xticks([])
        axs[0, 0].set_yticks([])


        sns.scatterplot(ax=axs[1,0],data=Data_Struct, x='tSNE-1 perp10', y='tSNE-2 perp10', hue='Population',legend = False)#.set(title='PCA')
        axs[1, 0].set_title('Perplexity = '+str(perplexity_values[1]))
        axs[1, 0].set_xlabel('tSNE-1')
        axs[1, 0].set_ylabel('tSNE-2')
        axs[1, 0].set_xticks([])
        axs[1, 0].set_yticks([])

        sns.scatterplot(ax=axs[2,0],data=Data_Struct, x='tSNE-1 perp30', y='tSNE-2 perp30', hue='Population',legend = False)#.set(title='PCA')
        axs[2, 0].set_title('Perplexity = '+str(perplexity_values[2]))
        axs[2, 0].set_xlabel('tSNE-1')
        axs[2, 0].set_ylabel('tSNE-2')
        axs[2, 0].set_xticks([])
        axs[2, 0].set_yticks([])

        sns.scatterplot(ax=axs[3,0],data=Data_Struct, x='tSNE-1 perp50', y='tSNE-2 perp50', hue='Population',legend = False)#.set(title='PCA')
        axs[3, 0].set_title('Perplexity = '+str(perplexity_values[3]))
        axs[3, 0].set_xlabel('tSNE-1')
        axs[3, 0].set_ylabel('tSNE-2')
        axs[3, 0].set_xticks([])
        axs[3, 0].set_yticks([])

        #UMAP section

                #n_neighbors_nums=(5,10,15,20)
                #mindists=(0.001,0.01,0.1,0.5)

        #Column1
        sns.scatterplot(ax=axs[0,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.001', y='UMAP-2 numn5 mindist0.001', hue='Population',legend = False)
        axs[0, 1].set_title('UMAP / n_neighbours = ' + str(n_neighbors_nums[0]))
        axs[0, 1].set_xlabel('UMAP-1')
        axs[0, 1].set_ylabel('min_dist = ' + str(mindists[0]))
        axs[0, 1].set_xticks([])
        axs[0, 1].set_yticks([])

        sns.scatterplot(ax=axs[1,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.01', y='UMAP-2 numn5 mindist0.01', hue='Population',legend = False)
        axs[1, 1].set_xlabel('UMAP-1')
        axs[1, 1].set_ylabel('min_dist = ' + str(mindists[1]))
        axs[1, 1].set_xticks([])
        axs[1, 1].set_yticks([])

        sns.scatterplot(ax=axs[2,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.1', y='UMAP-2 numn5 mindist0.1', hue='Population',legend = False)
        axs[2, 1].set_xlabel('UMAP-1')
        axs[2, 1].set_ylabel('min_dist = ' + str(mindists[2]))
        axs[2, 1].set_xticks([])
        axs[2, 1].set_yticks([])

        sns.scatterplot(ax=axs[3,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.5', y='UMAP-2 numn5 mindist0.5', hue='Population',legend = False)
        axs[3, 1].set_xlabel('UMAP-1')
        axs[3, 1].set_ylabel('min_dist = ' + str(mindists[3]))
        axs[3, 1].set_xticks([])
        axs[3, 1].set_yticks([])

        #Column2
        sns.scatterplot(ax=axs[0,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.001', y='UMAP-2 numn10 mindist0.001', hue='Population',legend = False)
        axs[0, 2].set_title('n_neighbours = ' + str(n_neighbors_nums[1]))
        axs[0, 2].set_xlabel('UMAP-1')
        axs[0, 2].set_ylabel('UMAP-2')
        axs[0, 2].set_xticks([])
        axs[0, 2].set_yticks([])

        sns.scatterplot(ax=axs[1,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.01', y='UMAP-2 numn10 mindist0.01', hue='Population',legend = False)
        axs[1, 2].set_xlabel('UMAP-1')
        axs[1, 2].set_ylabel('UMAP-2')
        axs[1, 2].set_xticks([])
        axs[1, 2].set_yticks([])

        sns.scatterplot(ax=axs[2,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.1', y='UMAP-2 numn10 mindist0.1', hue='Population',legend = False)
        axs[2, 2].set_xlabel('UMAP-1')
        axs[2, 2].set_ylabel('UMAP-2')
        axs[2, 2].set_xticks([])
        axs[2, 2].set_yticks([])

        sns.scatterplot(ax=axs[3,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.5', y='UMAP-2 numn10 mindist0.5', hue='Population',legend = False)
        axs[3, 2].set_xlabel('UMAP-1')
        axs[3, 2].set_ylabel('UMAP-2')
        axs[3, 2].set_xticks([])
        axs[3, 2].set_yticks([])

        #Column3
        sns.scatterplot(ax=axs[0,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.001', y='UMAP-2 numn15 mindist0.001', hue='Population',legend = False)
        axs[0, 3].set_title('n_neighbours = ' + str(n_neighbors_nums[2]))
        axs[0, 3].set_xlabel('UMAP-1')
        axs[0, 3].set_ylabel('UMAP-2')
        axs[0, 3].set_xticks([])
        axs[0, 3].set_yticks([])

        sns.scatterplot(ax=axs[1,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.01', y='UMAP-2 numn15 mindist0.01', hue='Population',legend = False)
        axs[1, 3].set_xlabel('UMAP-1')
        axs[1, 3].set_ylabel('UMAP-2')
        axs[1, 3].set_xticks([])
        axs[1, 3].set_yticks([])

        sns.scatterplot(ax=axs[2,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.1', y='UMAP-2 numn15 mindist0.1', hue='Population',legend = False)
        axs[2, 3].set_xlabel('UMAP-1')
        axs[2, 3].set_ylabel('UMAP-2')
        axs[2, 3].set_xticks([])
        axs[2, 3].set_yticks([])

        sns.scatterplot(ax=axs[3,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.5', y='UMAP-2 numn15 mindist0.5', hue='Population',legend = False)
        axs[3, 3].set_xlabel('UMAP-1')
        axs[3, 3].set_ylabel('UMAP-2')
        axs[3, 3].set_xticks([])
        axs[3, 3].set_yticks([])

        #Column4
        sns.scatterplot(ax=axs[0,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.001', y='UMAP-2 numn20 mindist0.001', hue='Population',legend = False)
        axs[0, 4].set_title('n_neighbours = ' + str(n_neighbors_nums[3]))
        axs[0, 4].set_xlabel('UMAP-1')
        axs[0, 4].set_ylabel('UMAP-2')
        axs[0, 4].set_xticks([])
        axs[0, 4].set_yticks([])

        sns.scatterplot(ax=axs[1,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.01', y='UMAP-2 numn20 mindist0.01', hue='Population',legend = False)
        axs[1, 4].set_xlabel('UMAP-1')
        axs[1, 4].set_ylabel('UMAP-2')
        axs[1, 4].set_xticks([])
        axs[1, 4].set_yticks([])

        sns.scatterplot(ax=axs[2,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.1', y='UMAP-2 numn20 mindist0.1', hue='Population',legend = False)
        axs[2, 4].set_xlabel('UMAP-1')
        axs[2, 4].set_ylabel('UMAP-2')
        axs[2, 4].set_xticks([])
        axs[2, 4].set_yticks([])

        sns.scatterplot(ax=axs[3,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.5', y='UMAP-2 numn20 mindist0.5', hue='Population')
        axs[3, 4].set_xlabel('UMAP-1')
        axs[3, 4].set_ylabel('UMAP-2')
        axs[3, 4].set_xticks([])
        axs[3, 4].set_yticks([])

        plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
        plt.savefig(filename[:-4]+'Top'+str(n_pc)+'PCs.png',format='png',dpi=100,transparent = False,facecolor='white')


        

In [ ]:
fig, axs = plt.subplots(4, 5,figsize=(25, 20))

#axs[1,1] = Data_Struct.plot.scatter(x='X1',y='X2',c='Population')


fig.suptitle("input:"+file_list+' / Top '+str(n_pc)+'PCs are used ('+str(round(expl_df['cumulative_expl'][n_pc-1],1))+'%)', fontsize=14)

sns.scatterplot(ax=axs[0,0],data=Data_Struct, x='tSNE-1 perp5', y='tSNE-2 perp5', hue='Population',legend = False)#.set(title='PCA')
axs[0, 0].set_title('tSNE / Perplexity = '+str(perplexity_values[0]))
axs[0, 0].set_xlabel('tSNE-1')
axs[0, 0].set_ylabel('tSNE-2')
axs[0, 0].set_xticks([])
axs[0, 0].set_yticks([])


sns.scatterplot(ax=axs[1,0],data=Data_Struct, x='tSNE-1 perp10', y='tSNE-2 perp10', hue='Population',legend = False)#.set(title='PCA')
axs[1, 0].set_title('Perplexity = '+str(perplexity_values[1]))
axs[1, 0].set_xlabel('tSNE-1')
axs[1, 0].set_ylabel('tSNE-2')
axs[1, 0].set_xticks([])
axs[1, 0].set_yticks([])

sns.scatterplot(ax=axs[2,0],data=Data_Struct, x='tSNE-1 perp30', y='tSNE-2 perp30', hue='Population',legend = False)#.set(title='PCA')
axs[2, 0].set_title('Perplexity = '+str(perplexity_values[2]))
axs[2, 0].set_xlabel('tSNE-1')
axs[2, 0].set_ylabel('tSNE-2')
axs[2, 0].set_xticks([])
axs[2, 0].set_yticks([])

sns.scatterplot(ax=axs[3,0],data=Data_Struct, x='tSNE-1 perp50', y='tSNE-2 perp50', hue='Population',legend = False)#.set(title='PCA')
axs[3, 0].set_title('Perplexity = '+str(perplexity_values[3]))
axs[3, 0].set_xlabel('tSNE-1')
axs[3, 0].set_ylabel('tSNE-2')
axs[3, 0].set_xticks([])
axs[3, 0].set_yticks([])

#UMAP section

        #n_neighbors_nums=(5,10,15,20)
        #mindists=(0.001,0.01,0.1,0.5)
        
#Column1
sns.scatterplot(ax=axs[0,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.001', y='UMAP-2 numn5 mindist0.001', hue='Population',legend = False)
axs[0, 1].set_title('UMAP / n_neighbours = ' + str(n_neighbors_nums[0]))
axs[0, 1].set_xlabel('UMAP-1')
axs[0, 1].set_ylabel('min_dist = ' + str(mindists[0]))
axs[0, 1].set_xticks([])
axs[0, 1].set_yticks([])

sns.scatterplot(ax=axs[1,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.01', y='UMAP-2 numn5 mindist0.01', hue='Population',legend = False)
axs[1, 1].set_xlabel('UMAP-1')
axs[1, 1].set_ylabel('min_dist = ' + str(mindists[1]))
axs[1, 1].set_xticks([])
axs[1, 1].set_yticks([])

sns.scatterplot(ax=axs[2,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.1', y='UMAP-2 numn5 mindist0.1', hue='Population',legend = False)
axs[2, 1].set_xlabel('UMAP-1')
axs[2, 1].set_ylabel('min_dist = ' + str(mindists[2]))
axs[2, 1].set_xticks([])
axs[2, 1].set_yticks([])

sns.scatterplot(ax=axs[3,1],data=Data_Struct, x='UMAP-1 numn5 mindist0.5', y='UMAP-2 numn5 mindist0.5', hue='Population',legend = False)
axs[3, 1].set_xlabel('UMAP-1')
axs[3, 1].set_ylabel('min_dist = ' + str(mindists[3]))
axs[3, 1].set_xticks([])
axs[3, 1].set_yticks([])

#Column2
sns.scatterplot(ax=axs[0,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.001', y='UMAP-2 numn10 mindist0.001', hue='Population',legend = False)
axs[0, 2].set_title('n_neighbours = ' + str(n_neighbors_nums[1]))
axs[0, 2].set_xlabel('UMAP-1')
axs[0, 2].set_ylabel('UMAP-2')
axs[0, 2].set_xticks([])
axs[0, 2].set_yticks([])

sns.scatterplot(ax=axs[1,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.01', y='UMAP-2 numn10 mindist0.01', hue='Population',legend = False)
axs[1, 2].set_xlabel('UMAP-1')
axs[1, 2].set_ylabel('UMAP-2')
axs[1, 2].set_xticks([])
axs[1, 2].set_yticks([])

sns.scatterplot(ax=axs[2,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.1', y='UMAP-2 numn10 mindist0.1', hue='Population',legend = False)
axs[2, 2].set_xlabel('UMAP-1')
axs[2, 2].set_ylabel('UMAP-2')
axs[2, 2].set_xticks([])
axs[2, 2].set_yticks([])

sns.scatterplot(ax=axs[3,2],data=Data_Struct, x='UMAP-1 numn10 mindist0.5', y='UMAP-2 numn10 mindist0.5', hue='Population',legend = False)
axs[3, 2].set_xlabel('UMAP-1')
axs[3, 2].set_ylabel('UMAP-2')
axs[3, 2].set_xticks([])
axs[3, 2].set_yticks([])

#Column3
sns.scatterplot(ax=axs[0,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.001', y='UMAP-2 numn15 mindist0.001', hue='Population',legend = False)
axs[0, 3].set_title('n_neighbours = ' + str(n_neighbors_nums[2]))
axs[0, 3].set_xlabel('UMAP-1')
axs[0, 3].set_ylabel('UMAP-2')
axs[0, 3].set_xticks([])
axs[0, 3].set_yticks([])

sns.scatterplot(ax=axs[1,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.01', y='UMAP-2 numn15 mindist0.01', hue='Population',legend = False)
axs[1, 3].set_xlabel('UMAP-1')
axs[1, 3].set_ylabel('UMAP-2')
axs[1, 3].set_xticks([])
axs[1, 3].set_yticks([])

sns.scatterplot(ax=axs[2,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.1', y='UMAP-2 numn15 mindist0.1', hue='Population',legend = False)
axs[2, 3].set_xlabel('UMAP-1')
axs[2, 3].set_ylabel('UMAP-2')
axs[2, 3].set_xticks([])
axs[2, 3].set_yticks([])

sns.scatterplot(ax=axs[3,3],data=Data_Struct, x='UMAP-1 numn15 mindist0.5', y='UMAP-2 numn15 mindist0.5', hue='Population',legend = False)
axs[3, 3].set_xlabel('UMAP-1')
axs[3, 3].set_ylabel('UMAP-2')
axs[3, 3].set_xticks([])
axs[3, 3].set_yticks([])

#Column4
sns.scatterplot(ax=axs[0,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.001', y='UMAP-2 numn20 mindist0.001', hue='Population',legend = False)
axs[0, 4].set_title('n_neighbours = ' + str(n_neighbors_nums[3]))
axs[0, 4].set_xlabel('UMAP-1')
axs[0, 4].set_ylabel('UMAP-2')
axs[0, 4].set_xticks([])
axs[0, 4].set_yticks([])

sns.scatterplot(ax=axs[1,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.01', y='UMAP-2 numn20 mindist0.01', hue='Population',legend = False)
axs[1, 4].set_xlabel('UMAP-1')
axs[1, 4].set_ylabel('UMAP-2')
axs[1, 4].set_xticks([])
axs[1, 4].set_yticks([])

sns.scatterplot(ax=axs[2,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.1', y='UMAP-2 numn20 mindist0.1', hue='Population',legend = False)
axs[2, 4].set_xlabel('UMAP-1')
axs[2, 4].set_ylabel('UMAP-2')
axs[2, 4].set_xticks([])
axs[2, 4].set_yticks([])

sns.scatterplot(ax=axs[3,4],data=Data_Struct, x='UMAP-1 numn20 mindist0.5', y='UMAP-2 numn20 mindist0.5', hue='Population')
axs[3, 4].set_xlabel('UMAP-1')
axs[3, 4].set_ylabel('UMAP-2')
axs[3, 4].set_xticks([])
axs[3, 4].set_yticks([])

plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
plt.savefig('Deneme.png',format='png',dpi=100,transparent = False,facecolor='white')
